In [19]:
import xmltodict 
import pandas as pd
from os import listdir
import os


### COMPOSICIÓN DEL XML

<presentacion>
    <periodo>…</periodo>
    <nroPresentacion>…</nroPresentacion>
    <fechaPresentacion>…</fechaPresentacion>
    <empleado>…</empleado>
    <cargasFamilia>…</cargasFamilia>
    <ganLiqOtrosEmpEnt>…</ganLiqOtrosEmpEnt>
    <deducciones>…</deducciones>
    <retPerPagos>…</retPerPagos>
    <ajustes>…</ajustes>
    <datosAdicionales>…</datosAdicionales>
<presentacion>

In [22]:
def apertura_deducciones_por_mes(empleado, tipo_deduccion, mes_desde, mes_hasta, monto_mensual):
    '''
    Esta funcion devuelve un df con la cantidad de registros necesarios, segun el intervalo desde hasta (periodo) por cada deducción.
    '''
    datos_572 = [[empleado, tipo_deduccion, monto_mensual]]
    df_datos_572 = pd.DataFrame(datos_572, columns=["cuil","codigo", "importe"])

    cant_filas = (mes_hasta - mes_desde + 1)

    df_datos_572 = pd.concat([df_datos_572]*cant_filas, ignore_index=True)

    nueva_columna = []

    for i in range(14):
        if mes_desde <= i <= mes_hasta: 
            nueva_columna.append(i)

    for index, fila in df_datos_572.iterrows():
        mes = mes_desde
        if mes <= mes_hasta:
            df_datos_572["Mes"] = nueva_columna
            mes = mes_desde + 1

    return df_datos_572


In [24]:
montos_cargas = {
    1 :     19621.44, 
    3 :     9895.16, 
    30 :    9895.16, 
    31 :    19790.33, 
    32:     19790.33,
    51:     19621.44, # VER CON VALE
    103:    0,
}

prefijo = "Q-CargFam-"

def cargas_por_mes(xml):

    empleado = xml["presentacion"]["empleado"]["cuit"]
    cargas = xml["presentacion"]["cargasFamilia"]["cargaFamilia"]
    df_por_mes_cargas = pd.DataFrame([], columns=["cuil", "codigo", "importe", "mes"])
        
    if isinstance(cargas, dict):
        cargas = [cargas]

    for i in cargas:
        parentesco = int(i["parentesco"])
        tipo_deduccion_carga = i["parentesco"]
        porcentaje_deduccion = int(i["porcentajeDeduccion"])
        mes_desde = int(i["mesDesde"])
        mes_hasta = int(i["mesHasta"])

        monto_mensual = montos_cargas[parentesco] * porcentaje_deduccion / 100

        df = apertura_deducciones_por_mes(empleado, prefijo + tipo_deduccion_carga, mes_desde, mes_hasta, monto_mensual)
        df_por_mes_cargas = pd.concat([df_por_mes_cargas, df], ignore_index=True)  

    return(df_por_mes_cargas)



In [33]:
CODIGOS_DEDUCC_MENSUAL = [1, 3, 4, 5, 8, 11, 21, 22, 32] #solamente dejar los que necesito que se procesen mensualmente
prefijo = "Q-Deduc-"

def deducciones_totales_por_mes(xml):

    empleado = xml["presentacion"]["empleado"]["cuit"]
    deducciones = xml["presentacion"]["deducciones"]["deduccion"]
    df_por_mes = pd.DataFrame([], columns=["cuil", "codigo", "importe", "mes"])
    
    if isinstance(deducciones, dict):
        deducciones = [deducciones]
    
    for i in deducciones:
        tipo_deduccion = int(i["@tipo"])
        if tipo_deduccion in CODIGOS_DEDUCC_MENSUAL:
            periodo = i["periodos"]["periodo"]
            
            #compruebo si es un diccionario y lo paso a listsa
            if isinstance(periodo, dict):
                periodo = [periodo]
            
            #para cada vaalor, determino los valores, ya sea que tengo una deduccion o muchas
            for j in periodo: 
                mes_desde = int(j["@mesDesde"])
                mes_hasta = int(j["@mesHasta"])
                monto_mensual = float(j["@montoMensual"])
                df = apertura_deducciones_por_mes(empleado, prefijo + str(tipo_deduccion), mes_desde, mes_hasta, monto_mensual)
                df_por_mes = pd.concat([df_por_mes, df], ignore_index=True)            
        else:
            monto_anual = float(i["montoTotal"])
            df = apertura_deducciones_por_mes(empleado, prefijo + str(tipo_deduccion), 13, 13, monto_anual)
            df_por_mes = pd.concat([df_por_mes, df], ignore_index=True)

    return df_por_mes


In [28]:
prefijo = 'Q-OE-'
    
def df_otro_empleador(xml):

    empleado = xml["presentacion"]["empleado"]["cuit"]
    ingresos_oe = xml["presentacion"]["ganLiqOtrosEmpEnt"]["empEnt"]["ingresosAportes"]["ingAp"]
    df_otro_empleador = pd.DataFrame([], columns=["cuil", "codigo", "importe", "mes"])

    if isinstance(ingresos_oe, dict):
        ingresos_oe = [ingresos_oe]

    for mes in ingresos_oe:
        for k in list(mes.keys()):
            if k == "@mes":
                continue 
            valor = float(mes[k])
            if valor != 0:
                datos_df_oe = [empleado, prefijo + k, valor, int(mes["@mes"])]
                df_oe_new = pd.DataFrame([datos_df_oe], columns=["cuil", "codigo", "importe", "mes"])
                df_otro_empleador = pd.concat([df_otro_empleador, df_oe_new], ignore_index=True)
        
    return(df_otro_empleador)


In [29]:
prefijo = 'Q-RetPerPagCu-'
def df_ret_per_pago(xml):

    empleado = xml["presentacion"]["empleado"]["cuit"]
    rpp = xml["presentacion"]["retPerPagos"]["retPerPago"]
    df_por_rrp = pd.DataFrame([], columns=["cuil", "codigo", "importe", "mes"])
    
    if isinstance(rpp, dict):
        rpp = [rpp]
    
    for i in rpp:
        tipo_deduccion = int(i["@tipo"])            
        monto_mensual = float(i["montoTotal"])
        datos_df_rpp = [empleado, prefijo + str(tipo_deduccion), monto_mensual, 13]
        df = pd.DataFrame([datos_df_rpp], columns=["cuil", "codigo", "importe", "mes"])
        df_por_rrp = pd.concat([df_por_rrp, df], ignore_index=True) 

    return df_por_rrp

In [30]:
carpeta_xml = "../../bases-ganancias-2022/in/formularios572/"

In [34]:
def deducciones_finales(carpeta):

    listdir(carpeta_xml)
    df_deducciones = pd.DataFrame()

    for a in listdir(carpeta_xml):
        path = carpeta_xml + "\\" + a

        with open(path) as f:
            xml = xmltodict.parse(f.read())

            if "deducciones" in xml["presentacion"]:
                df_deducc_mes_por_empleado = deducciones_totales_por_mes(xml)
                df_deducciones = pd.concat([df_deducc_mes_por_empleado, df_deducciones], ignore_index=True)

            if "cargasFamilia" in xml["presentacion"]:
                df_cargas_por_empleado_mes = cargas_por_mes(xml)
                df_deducciones = pd.concat([df_cargas_por_empleado_mes, df_deducciones], ignore_index=True)

            if "ganLiqOtrosEmpEnt" in xml["presentacion"]:
                df_ing_aport_otro_empleador = df_otro_empleador(xml)
                df_deducciones = pd.concat([df_ing_aport_otro_empleador, df_deducciones], ignore_index=True)

            if "retPerPagos" in xml["presentacion"]:
                df_rpp = df_ret_per_pago(xml)
                df_deducciones= pd.concat([df_rpp, df_deducciones], ignore_index=True)
                
    return(df_deducciones)


In [35]:
df = deducciones_finales(carpeta_xml)
df


KeyError: 51

In [21]:
#df.to_parquet('../../bases-ganancias-2022/middle/572web.parquet', compression='brotli')

In [17]:
import pandas as pd
df = pd.read_parquet('../../bases-ganancias-2022/middle/572web.parquet')

display(list(df['Deduccion'].unique()))
display(list(df['Descripción'].unique()))

['C3',
 'D7',
 'D8',
 'RPP-27',
 'RPP-28',
 'D1',
 'D2',
 None,
 'C1',
 'D3',
 'D4',
 'D21',
 'D23',
 'D24',
 'D99',
 'D22',
 'C31',
 'D9',
 'C32']

[None, 'OE-obraSoc', 'OE-segSoc', 'OE-ganBrut', 'OE-sac']